In [1]:
import os, gzip
import matplotlib.pyplot as plt
import pysam

import sys
sys.path.append("../../../src/")
import lakeview as lv

plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['figure.dpi'] = 72

In [2]:
%matplotlib widget
plt.ioff()

In [3]:
CHROMOSOME = "chr14"
START = 104586347
END = 107043718
REGION = f"{CHROMOSOME}:{START}-{END}"
PACBIO_BAM_URL = "ftp://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/HG002_NA24385_son/PacBio_CCS_15kb_20kb_chemistry2/GRCh38/HG002.SequelII.merged_15kb_20kb.pbmm2.GRCh38.haplotag.10x.bam"
PACBIO_BAI_URL = "ftp://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/HG002_NA24385_son/PacBio_CCS_15kb_20kb_chemistry2/GRCh38/HG002.SequelII.merged_15kb_20kb.pbmm2.GRCh38.haplotag.10x.bam.bai"
PACBIO_BAM_PATH = "data/PacBio_CCS.bam"
PACBIO_BAI_PATH = "data/PacBio_CCS.bam.bai"


PACBIO_BAM_URL = "ftp://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/HG002_NA24385_son/PacBio_CCS_15kb/GRCh38_no_alt_analysis/HG002.15kb.Sequel.pbmm2.GRCh38.whatshap.haplotag.RTG.10x.trio.bam"
PACBIO_BAI_URL = "ftp://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/HG002_NA24385_son/PacBio_CCS_15kb/GRCh38_no_alt_analysis/HG002.15kb.Sequel.pbmm2.GRCh38.whatshap.haplotag.RTG.10x.trio.bam.bai"

if not os.path.isfile(PACBIO_BAM_PATH):
    bam_data = pysam.view(
        "-X",
        "-b",
        PACBIO_BAM_URL,
        PACBIO_BAI_URL,
        REGION,
    )
    with open(PACBIO_BAM_PATH, "wb") as f:
        f.write(bam_data)
if not os.path.isfile(PACBIO_BAI_PATH):
    pysam.index(PACBIO_BAM_PATH)

In [4]:
pacbio_painter = lv.SequenceAlignment.from_file(PACBIO_BAM_PATH, "rb")

In [5]:
with gzip.open("data/gencode.v40.annotation.gtf.gz", "rt") as f:
    gencode_painter = lv.GeneAnnotation.from_file(
        file_handle=f,
        format="gtf",
        sequence_name=CHROMOSOME,
        start=START,
        end=END,
    )

In [10]:
gv = lv.GenomeViewer(width=12, height=12, xlim=(105679000, 105776000))
gv.add_track(pacbio_painter.draw_pileup, 1, show_mismatches=False)
gv.add_track(
    pacbio_painter.draw_alignment,
    8,
    show_mismatches=False,
    order=lambda seg: -seg.query_alignment_length,
    max_group_depth=50,
)
gv.add_track(
    gencode_painter.draw_genes,
    0.5,
    labels=[g.attributes.get("gene_name", "") for g in gencode_painter.genes],
    allow_overlaps=True,
)
gv.init_app()

In [11]:
gv

AppLayout(children=(VBox(children=(HBox(children=(Text(value='104,572,680 - 106,883,718', placeholder=''), But…

In [16]:
gv = lv.GenomeViewer(width=12, height=12, xlim=(105679000, 105776000))
gv.add_track(pacbio_painter.draw_pileup, 1, show_mismatches=False)
gv.add_track(
    pacbio_painter.draw_alignment,
    8,
    show_mismatches=False,
    order=lambda seg: -seg.query_alignment_length,
    max_group_depth=50,
)
gv.init_app()

In [17]:
gv

AppLayout(children=(VBox(children=(HBox(children=(Text(value='104,572,680 - 106,883,718', placeholder=''), But…

In [11]:
gv = lv.GenomeViewer(width=4, height=4, use_ipympl=True)
gv.add_track(lambda ax: ax.plot([1, 2]))
gv.init_app()

In [12]:
gv

AppLayout(children=(VBox(children=(HBox(children=(Text(value='0 - 1', placeholder=''), Button(description='Go'…

In [74]:
import ipywidgets as widgets
from IPython.display import display

In [82]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot([1, 2])
app = widgets.AppLayout(
    center=fig.canvas,
    footer=widgets.Button(icon='check'),
    pane_heights=[0, 6, 1]
)

class C:
    def __init__(self, app):
        self.app = app
    
    def _ipython_display_(self):
        return display(self.app)
    
gv = C(app)

In [83]:
gv

AppLayout(children=(Button(icon='check', layout=Layout(grid_area='footer'), style=ButtonStyle()), Canvas(layou…

In [62]:
?gv.figure.canvas.draw

Signature: gv.figure.canvas.draw()
Docstring:
Render the `.Figure`.

It is important that this method actually walk the artist tree
even if not output is produced because this will trigger
deferred work (like computing limits auto-limits and tick
values) that users may want access to before saving to disk.
File:      ~/opt/anaconda3/envs/dev/lib/python3.9/site-packages/matplotlib/backends/backend_webagg_core.py
Type:      method


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
painter.draw_alignment(
    ax,
    show_mismatches=False,
    order=lambda seg: -seg.query_alignment_length,
)

NameError: name 'painter' is not defined

In [ ]:
fig

In [ ]:
gencode_painter.genes[:10]